In [1]:
%matplotlib inline
import xarray as xr
import numpy as np 
import glob 
import os 
import pickle
from tdigest import TDigest
import matplotlib
import matplotlib.pyplot as plt

os.chdir("/home/bsc32/bsc32263/git/one_pass")
from one_pass.opa import *
from one_pass.opa import Opa

In [5]:
## imagine this is the gsv request ## 
## include your data here (eg. gsv data, nextGEMS, must be on lat lon grid) ## 
filePathData = "/esarchive/scratch/alacima/projects/destination_earth/icon/*.nc"
fileList = glob.glob(filePathData) 
fileList.sort() 
data = xr.open_dataset(fileList[0]) # open dataset
uas = np.squeeze(data.uas)


In [6]:
pass_dic = {"stat": "mean",
"stat_freq": "daily",
"output_freq": "daily",
"time_step": 60,
"variable": "tas",
"save": True,
"checkpoint":True,
"checkpoint_in_file": "/home/bsc32/bsc32263/git/data/checkpoint_in_mean_daily_tas.nc",
"checkpoint_out_file": "/home/bsc32/bsc32263/git/data/checkpoint_in_mean_daily_tas.nc",
"out_file": "/home/bsc32/bsc32263/git/data"}

In [7]:
daily_mean = Opa("config.yml")
#daily_mean = Opa(pass_dic)

In [9]:
daily_mean.statistic

'mean'

In [10]:
file_name = "testfile"

with open(file_name, 'wb') as file: 
    pickle.dump(daily_mean, file)

In [12]:
# reloading the object 
file_name = "testfile"

with open(file_name, 'rb') as file: 
    daily_mean2 = pickle.load(file)

In [13]:
daily_mean2.statistic

'mean'

In [4]:
#file_path = "/home/bsc32/bsc32263/git/one_pass/config.yml"


for i in range(0, 1, 1): 

    ds = data.isel(time=slice(i,i+1)) # extract moving window of tas variable
    dm = daily_mean.compute(ds)

dm

KeyError: 'output_freq'

In [5]:
daily_mean.statistic


'mean'

In [22]:
daily_mean.count

2

In [13]:
from one_pass.util import load_yaml

file_path = "/home/bsc32/bsc32263/git/one_pass/config.yml"
#file_path = "/home/bsc32/bsc32263/git/gsv_interface/request.yaml"
config = load_yaml(file_path)

config

{'levtype': 'sfc',
 'date': ['20050401'],
 'time': ['0000',
  '0100',
  '0200',
  '0300',
  '0400',
  '0500',
  '0600',
  '0700',
  '0800',
  '0900',
  '1000',
  '1100',
  '1200',
  '1300',
  '1400',
  '1500',
  '1600',
  '1700',
  '1800',
  '1900',
  '2000',
  '2100',
  '2200',
  '2300'],
 'time_step': 60,
 'step': '0',
 'param': ['165', '166', '167'],
 'stat': ['mean', 'mean', 'std'],
 'stat_freq': ['hourly', 'daily', 'daily'],
 'file_path_save': '/home/bsc32/bsc32263/git/data',
 'stats': {'mean': {'var_id': '165',
   'stat_freq': ['daily'],
   'time_step': ['0300']},
  'sdev': {'var_id': '160', 'stat_freq': ['daily'], 'time_step': ['0300']}}}

In [19]:
np.size(config["stats"])

1

In [ ]:
# IMPORT PACKAGES 

# GET GSV DATA (SAY TWO HOURS AVALIABLE)

daily_std = Opa(statistic = "std", stat_freq = "daily", output_freq = "daily", save = False, variable = "tas", config_path = file_path)
dm = daily_std.compute(ds)

daily_std = Opa("file or seperate dictionary") 
daily_std.initialise_from_file(file.yaml)

# initialise needs to contain a checkpoint file "checkpoint_opa_tas_##.nc"

# changes: 
# want this to come from a config file 
# don't want to have to run two lines, as we can't initalise every time 
# OR 
# initialise everytime but inside .compute also looks for files saved to disk